### Vehicle Detection Project -- Classification Part

Zikri Bayraktar, Ph.D.

This project has two parts ( in this notebook, we will focus on Part 1 ):
1. Train a classifier to identify cars on an image of a road.
2. Create a pipeline to detect vehicles in a video stream.

#### Part 1: Train a classifier to identify cars vs. non-cars (i.e. background)

In [26]:
import numpy as np
import cv2
import glob
import os
import time

from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from skimage.feature import hog

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [27]:
# Read in all available car images for training classifier:

vehicle_list=[]
for foldername in os.listdir('folder_vehicles'):
    for filename in glob.glob('folder_vehicles/'+str(foldername)+'/*.png'):
        vehicle_list.append(filename)

In [28]:
# Read in all available nonCar images for training classifier:

non_vehicle_list=[]
for filename in glob.glob('folder_non_vehicles/*.png'):
    non_vehicle_list.append(filename)

#### Some useful functions from lectures

In [29]:
# Define a function to compute binned color features.
# This function simply resizes image to a smaller size and creates a 1D feature vector to return.
# By shrinking the image, we are reducing the dimensions while still maintaining some relavant 
# features to help us identify cars visually.
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    #print('bin_spatial ' + str(max(features)))   --> range of [0,1]
    return features

In [30]:
# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the RGB channels separately
    c1hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    c2hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    c3hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Generating bin centers
    bin_edges = c1hist[1]
    bin_centers = (bin_edges[1:]  + bin_edges[0:len(bin_edges)-1])/2
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((c1hist[0], c2hist[0], c3hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    #return c1hist, c2hist, c3hist, bin_centers, hist_features
    #print('hist ' + str(hist_features))
    return hist_features

In [31]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_color_features(imgs, cspace='RGB', spatial_size=(32, 32), hist_bins=32, hist_range=(0, 256)):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        
        image = image*255  
        
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        else: feature_image = np.copy(image)      
        
        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #print('spatial ' + str(max(spatial_features)))
        # Apply color_hist() also with a color space option now
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
        #print('hist ' + str(max(hist_features)))
        # Append the new feature vector to the features list
        features.append(np.concatenate((spatial_features, hist_features)))
    # Return list of feature vectors
    return features

In [32]:
# Define a function to extract HOG features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_hog_features(imgs, cspace='RGB', orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        image=image*255
        
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        # Append the new feature vector to the features list
        features.append(hog_features)
    # Return list of feature vectors
    return features


In [33]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    draw_img = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(draw_img, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return draw_img

In [34]:
# Define a function to search for template matches
# and return a list of bounding boxes
def find_matches(img, template_list):
    # Define an empty list to take bbox coords
    bbox_list = []
    # Define matching method
    # Other options include: cv2.TM_CCORR_NORMED', 'cv2.TM_CCOEFF', 'cv2.TM_CCORR',
    #         'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED'
    method = cv2.TM_CCOEFF_NORMED
    # Iterate through template list
    for temp in template_list:
        # Read in templates one by one
        tmp = mpimg.imread(temp)
        # Use cv2.matchTemplate() to search the image
        result = cv2.matchTemplate(img, tmp, method)
        # Use cv2.minMaxLoc() to extract the location of the best match
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        # Determine a bounding box for the match
        w, h = (tmp.shape[1], tmp.shape[0])
        if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
            top_left = min_loc
        else:
            top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        # Append bbox position to list
        bbox_list.append((top_left, bottom_right))
        # Return the list of bounding boxes
        
    return bbox_list

In [35]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), block_norm= 'L2-Hys',
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), block_norm= 'L2-Hys',
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

In [36]:
from mpl_toolkits.mplot3d import Axes3D

## Function to plot 3D figures to visualize color spaces:
def plot3d(pixels, colors_rgb,
        axis_labels=list("RGB"), axis_limits=((0, 255), (0, 255), (0, 255))):
    """Plot pixels in 3D."""

    # Create figure and 3D axes
    fig = plt.figure(figsize=(8, 8))
    ax = Axes3D(fig)

    # Set axis limits
    ax.set_xlim(*axis_limits[0])
    ax.set_ylim(*axis_limits[1])
    ax.set_zlim(*axis_limits[2])

    # Set axis labels and sizes
    ax.tick_params(axis='both', which='major', labelsize=14, pad=8)
    ax.set_xlabel(axis_labels[0], fontsize=16, labelpad=16)
    ax.set_ylabel(axis_labels[1], fontsize=16, labelpad=16)
    ax.set_zlabel(axis_labels[2], fontsize=16, labelpad=16)

    # Plot pixel values with colors given in colors_rgb
    ax.scatter(
        pixels[:, :, 0].ravel(),
        pixels[:, :, 1].ravel(),
        pixels[:, :, 2].ravel(),
        c=colors_rgb.reshape((-1, 3)), edgecolors='none')

    return ax  # return Axes3D object for further manipulation

### Extract COLOR features:

In [65]:
# Extract Color Features:
spatial = 16
histbin = 32
colorspace = 'HSV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb


car_color_features = extract_color_features(vehicle_list, cspace=colorspace, spatial_size=(spatial, spatial),
                        hist_bins=histbin, hist_range=(0, 1.0))


notcar_color_features = extract_color_features(non_vehicle_list, cspace=colorspace, spatial_size=(spatial, spatial),
                        hist_bins=histbin, hist_range=(0, 1.0))

### Extract HOG Features:

In [79]:
orient = 9
pix_per_cell = 16
cell_per_block = 2
hog_channel = "ALL"  # Can be 0, 1, 2, or "ALL"

t=time.time()
car_hog_features = extract_hog_features(vehicle_list, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
notcar_hog_features = extract_hog_features(non_vehicle_list, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)

In [80]:
print(len(car_color_features))
print(len(car_hog_features))
print(len(notcar_color_features))
print(len(notcar_hog_features))

8792
8792
5068
5068


In [81]:
# Warning! Pay Attention!
# Note that car_all_features is a 'list'. Each element of this list represents the combined
# features of a single image. 

# Combine color and hog features for images labeled as CAR
car_all_features=[]
for colorf,hogf in zip(car_color_features,car_hog_features):
    car_all_features.append(np.concatenate((colorf, hogf)))

In [82]:
# Combine color and hog features for images labeled as NONCAR
notcar_all_features=[]
for colorf, hogf in zip(notcar_color_features, notcar_hog_features):
    notcar_all_features.append(np.concatenate((colorf,hogf)))

In [83]:
#print(notcar_all_features[0])
#print(car_all_features[0])

In [84]:
print(max(car_color_features[0]))

627.0


In [85]:
# Stack the CAR and NONCAR
X = np.vstack((car_all_features, notcar_all_features)).astype(np.float64)                        

In [86]:
# Stack the corresponding labels:
y = np.hstack((np.ones(len(car_all_features)), np.zeros(len(notcar_all_features))))

In [87]:
# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rand_state)

In [88]:
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)    

In [89]:
# Fit a per-column scaler only on the training data
X_scaler = StandardScaler().fit(X_train)
# Apply the scaler to X_train and X_test
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [90]:
X_train[0]

array([-0.87950479,  2.00008755, -0.57689187, ..., -0.07921256,
       -0.09748271, -0.17911283])

In [91]:
print('Using spatial binning of:',spatial, 'and', histbin,'histogram bins')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Train Accuracy of SVC = ', round(svc.score(X_train, y_train), 4))
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample


Using spatial binning of: 16 and 32 histogram bins
Feature vector length: 1836
4.03 Seconds to train SVC...
Train Accuracy of SVC =  1.0
Test Accuracy of SVC =  0.9931


In [26]:
t=time.time()
n_predict = 100
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these' 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

My SVC predicts:  [ 0.  0.  0.  1.  1.  0.  1.  1.  0.  0.  0.  1.  1.  0.  1.  1.  1.  1.
  1.  1.  0.  1.  0.  1.  1.  0.  1.  0.  0.  0.  1.  1.  1.  0.  1.  1.
  0.  1.  0.  0.  1.  1.  1.  0.  1.  1.  0.  1.  1.  1.  1.  1.  0.  0.
  1.  1.  0.  1.  0.  1.  1.  1.  1.  1.  1.  1.  0.  1.  1.  1.  0.  0.
  0.  0.  0.  1.  1.  0.  1.  0.  1.  0.  1.  0.  1.  1.  1.  1.  0.  1.
  0.  1.  1.  1.  0.  1.  1.  1.  1.  0.]
For theselabels:  [ 0.  0.  0.  1.  1.  0.  1.  1.  0.  0.  0.  1.  1.  0.  1.  1.  1.  1.
  1.  1.  0.  1.  0.  1.  1.  0.  1.  1.  0.  0.  1.  1.  1.  0.  1.  1.
  0.  1.  0.  0.  1.  1.  1.  0.  1.  1.  0.  1.  1.  1.  1.  1.  0.  0.
  1.  1.  0.  1.  0.  1.  1.  1.  1.  1.  1.  1.  0.  1.  1.  1.  0.  0.
  0.  0.  0.  1.  1.  0.  1.  0.  1.  0.  1.  0.  1.  1.  1.  1.  0.  1.
  0.  1.  1.  1.  0.  1.  1.  1.  1.  0.]
0.01651 Seconds to predict 100 labels with SVC


In [92]:
# Save model to a file
import pickle

# Save the model:
with open('model_HSV_hist_range.pickle', 'wb') as handle:
    pickle.dump(svc, handle)
    
# Save the scalar function used to normalize the data:
with open('scalar_HSV_hist_range.pickle', 'wb') as handle:
    pickle.dump(X_scaler, handle)
    

In [26]:
# Load a model from a pickle file:
with open('model_YCrCb.pickle', 'rb') as handle:
    inmodel = pickle.load(handle)

In [27]:
print('Test Accuracy of loaded model = ', round(inmodel.score(X_test, y_test), 4))

Test Accuracy of loaded model =  0.9957


In [28]:
# Some References:
# 1. Pickle file write/read:
# https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict
# 2. 